In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

Loading the data

In [2]:
#pickle_file = 'notMNIST.pickle'
pickle_file = 'notMNIST_clean.pickle'
data = {}
try:
    with open(pickle_file, 'rb') as f:
        data = pickle.load(f)
        train_dataset = data['train_dataset']
        train_labels = data['train_labels']
        valid_dataset = data['valid_dataset']
        valid_labels = data['valid_labels']
        test_dataset = data['test_dataset']
        test_labels = data['test_labels']
    del data
except Exception as e:
    print('Unable to load data from', pickle_file, ':', e)

In [3]:
IMAGE_SIZE = 28
NUM_LABELS = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, IMAGE_SIZE **2)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(NUM_LABELS) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (185307, 784) (185307, 10)
Validation set (8856, 784) (8856, 10)
Test set (8609, 784) (8609, 10)


In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE **2, NUM_LABELS]))
    biases = tf.Variable(tf.zeros([NUM_LABELS]))
  
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.360159
Training accuracy: 8.1%
Validation accuracy: 10.1%
Test accuracy: 9.5%
Loss at step 100: 2.304311
Training accuracy: 71.8%
Validation accuracy: 70.0%
Test accuracy: 77.0%
Loss at step 200: 1.871752
Training accuracy: 74.8%
Validation accuracy: 72.3%
Test accuracy: 79.1%
Loss at step 300: 1.636812
Training accuracy: 76.0%
Validation accuracy: 73.2%
Test accuracy: 79.7%
Loss at step 400: 1.476907
Training accuracy: 76.9%
Validation accuracy: 73.6%
Test accuracy: 80.1%
Loss at step 500: 1.357971
Training accuracy: 77.4%
Validation accuracy: 73.9%
Test accuracy: 80.3%
Loss at step 600: 1.264163
Training accuracy: 77.9%
Validation accuracy: 73.9%
Test accuracy: 80.5%
Loss at step 700: 1.187282
Training accuracy: 78.4%
Validation accuracy: 74.1%
Test accuracy: 80.7%
Loss at step 800: 1.122679
Training accuracy: 78.8%
Validation accuracy: 74.4%
Test accuracy: 81.0%


In [18]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, IMAGE_SIZE **2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(
    tf.truncated_normal([IMAGE_SIZE **2, NUM_LABELS]))
    biases = tf.Variable(tf.zeros([NUM_LABELS]))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.926405
Minibatch accuracy: 4.7%
Validation accuracy: 10.1%
Minibatch loss at step 500: 2.013081
Minibatch accuracy: 70.3%
Validation accuracy: 73.8%
Minibatch loss at step 1000: 1.220457
Minibatch accuracy: 76.6%
Validation accuracy: 75.1%
Minibatch loss at step 1500: 1.254385
Minibatch accuracy: 75.8%
Validation accuracy: 76.5%
Minibatch loss at step 2000: 1.024863
Minibatch accuracy: 76.6%
Validation accuracy: 76.9%
Minibatch loss at step 2500: 1.105963
Minibatch accuracy: 78.9%
Validation accuracy: 76.8%
Minibatch loss at step 3000: 0.798304
Minibatch accuracy: 82.0%
Validation accuracy: 77.6%
Test accuracy: 84.3%


Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [30]:
BATCH_SIZE = 128
HIDDEN_NODES = 1024

graph = tf.Graph()
def calculate_layer1(prev_layer, weights, biases):
    return tf.nn.relu(tf.matmul(prev_layer, weights) + biases)

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(BATCH_SIZE, IMAGE_SIZE**2))
    tf_train_labels = tf.placeholder(tf.float32, shape=(BATCH_SIZE, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights0 = tf.Variable(tf.truncated_normal([IMAGE_SIZE **2, HIDDEN_NODES]))
    biases0 = tf.Variable(tf.zeros([HIDDEN_NODES]))
    weights = tf.Variable(tf.truncated_normal([HIDDEN_NODES, NUM_LABELS]))
    biases = tf.Variable(tf.zeros([NUM_LABELS]))
    
    tf_train_layer1 = calculate_layer1(tf_train_dataset, weights0, biases0) 
    logits = tf.matmul(tf_train_layer1, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    
    tf_valid_layer1 = calculate_layer1(tf_valid_dataset, weights0, biases0) 
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_layer1, weights) + biases)
    tf_test_layer1 = calculate_layer1(tf_test_dataset, weights0, biases0)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_layer1, weights) + biases)

In [31]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 290.465881
Minibatch accuracy: 9.4%
Validation accuracy: 28.2%
Minibatch loss at step 500: 12.575353
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 8.426568
Minibatch accuracy: 83.6%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 6.857693
Minibatch accuracy: 84.4%
Validation accuracy: 79.6%
Minibatch loss at step 2000: 7.047446
Minibatch accuracy: 77.3%
Validation accuracy: 80.5%
Minibatch loss at step 2500: 6.418442
Minibatch accuracy: 79.7%
Validation accuracy: 80.3%
Minibatch loss at step 3000: 8.938875
Minibatch accuracy: 83.6%
Validation accuracy: 80.4%
Test accuracy: 87.4%
